In [3]:
import random
import numpy as np

class TSPGeneticAlgorithm:
    def __init__(self, tsp_graph, population_size):
        self.TSP = tsp_graph
        self.city_num = np.shape(self.TSP)[0]
        self.pop_size = population_size
        self.score = np.empty(self.pop_size)
        
        """Route will start from zero"""
        self.genotype_pool = np.zeros([self.pop_size, self.city_num+1], dtype = int)
        for i in range(self.pop_size):
            city_indices = np.array(range(1, self.city_num))
            np.random.shuffle(city_indices)
            self.genotype_pool[i][1:self.city_num] = city_indices
        
        self.best_score = 0
        self.best_route = np.empty(self.city_num+1)
            
    def score_function(self):
        for i in range(self.pop_size):
            tmp_score = 0
            for k in range(self.city_num):
                tmp_score += self.TSP[self.genotype_pool[i][k], self.genotype_pool[i][k+1]]
            self.score[i] = tmp_score
            
        #Update Best Score and Route
        self.best_score = np.amin(self.score)
        self.best_route = self.genotype_pool[np.argsort(self.score)[0]]
        print("Shortest Distance: ", self.best_score)
        print("Best Route: ", self.best_route)
        return self.score
    
    def parent_genes(self, number_of_parents):
        fitness = 1/self.score
        survival_probability = fitness/np.sum(fitness)
        parent_genotype = self.genotype_pool[np.random.choice(range(self.pop_size), number_of_parents, replace=False, p=survival_probability)]
        return parent_genotype
    
    def elitism(self, number_of_elites):
        return self.genotype_pool[np.argsort(self.score)[:number_of_elites]]

    def crossover(self, parent_gene_a, parent_gene_b):
        """
        INPUT: (1, gene_length)Numpy Ndarray: 2 Parent Genes
        OUTPUT: (1, gene_length)Numpy Ndarray: 2 Child Gene
        """
        def child(_a, _b):
            snippets = sorted(random.sample(range(1, self.city_num), 2))
            offspring_a = _a[snippets[0]:snippets[1]]
            offspring_b = _b
            offspring_b = np.setdiff1d(offspring_b, offspring_a)
            
            child_gene = offspring_b[:snippets[0]]
            child_gene = np.append(child_gene, offspring_a)
            child_gene = np.append(child_gene, offspring_b[snippets[0]:])
            child_gene = np.append(child_gene, 0)
            return child_gene
        
        child_genotype = child(parent_gene_a, parent_gene_b)
        child_genotype = np.stack((child_genotype, child(parent_gene_b, parent_gene_a)))
        return child_genotype
              
    def mutation(self, parent_gene):
        """
        INPUT: (1, gene_length)Numpy Ndarray: Parent Gene
        OUTPUT: (1, gene_length)Numpy Ndarray: Child Gene
        """
        snippets = random.sample(range(1, self.city_num), 2)
        child_gene = parent_gene
        child_gene[0][snippets[0]] = parent_gene[0][snippets[1]]
        child_gene[0][snippets[1]] = parent_gene[0][snippets[0]]
        return child_gene
    
    def optimization(self, iterations, number_of_elites, number_of_parents, crossover_rate = 0.7):
        for i in range(iterations):
            self.score_function()
            parents = self.parent_genes(number_of_parents)
            nextgen = self.elitism(number_of_elites)
            for j in range(int(self.pop_size*crossover_rate)):
                a_index, b_index = random.sample(range(number_of_parents), 2)
                tmp_children = self.crossover(parents[a_index], parents[b_index])
                nextgen = np.append(nextgen, tmp_children, axis=0)
            for j in range(self.pop_size - number_of_elites - int(self.pop_size * crossover_rate)):
                chosen_parent = random.sample(range(number_of_parents), 1)
                tmp_children = self.mutation(parents[chosen_parent])
                nextgen = np.append(nextgen, tmp_children, axis=0)
            self.genotype_pool= nextgen
                
        self.score_function()

In [4]:
graph = np.array([[0, 10, 15, 20], [5, 0, 9, 10], [6, 13, 0, 12], [8, 8, 9, 0]])
print(graph)
ga = TSPGeneticAlgorithm(graph, 20)
print(ga.genotype_pool)
ga.optimization(30, 5, 2)

[[ 0 10 15 20]
 [ 5  0  9 10]
 [ 6 13  0 12]
 [ 8  8  9  0]]
[[0 1 3 2 0]
 [0 2 3 1 0]
 [0 3 2 1 0]
 [0 1 2 3 0]
 [0 1 3 2 0]
 [0 1 3 2 0]
 [0 3 1 2 0]
 [0 3 1 2 0]
 [0 3 2 1 0]
 [0 1 2 3 0]
 [0 1 3 2 0]
 [0 2 1 3 0]
 [0 2 3 1 0]
 [0 2 3 1 0]
 [0 3 2 1 0]
 [0 3 1 2 0]
 [0 3 1 2 0]
 [0 1 2 3 0]
 [0 1 3 2 0]
 [0 3 2 1 0]]
Shortest Distance:  35.0
Best Route:  [0 1 3 2 0]
Shortest Distance:  35.0
Best Route:  [0 1 3 2 0]
Shortest Distance:  35.0
Best Route:  [0 1 3 2 0]
Shortest Distance:  35.0
Best Route:  [0 1 3 2 0]
Shortest Distance:  35.0
Best Route:  [0 1 3 2 0]
Shortest Distance:  35.0
Best Route:  [0 1 3 2 0]
Shortest Distance:  35.0
Best Route:  [0 1 3 2 0]
Shortest Distance:  35.0
Best Route:  [0 1 3 2 0]
Shortest Distance:  35.0
Best Route:  [0 1 3 2 0]
Shortest Distance:  35.0
Best Route:  [0 1 3 2 0]
Shortest Distance:  35.0
Best Route:  [0 1 3 2 0]
Shortest Distance:  35.0
Best Route:  [0 1 3 2 0]
Shortest Distance:  35.0
Best Route:  [0 1 3 2 0]
Shortest Distance:  35.0
Bes